In [3]:
%load_ext autoreload
%autoreload 2 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os

In [5]:
from models import GazeEstimation_ResNet18

# Configuraciones

Configuraciones para CUDA

In [ ]:
# Configuraciones para CUDA
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device a utilizar:",device)

# Analisis de datos

# Entrenamiento

Instanciación del modelo y prueba de dimensiones

In [6]:
# Instanciación del model
model = GazeEstimation_ResNet18(pretrained=True)

# Al ser un problema de regresión se usa MSELoss
criterion = nn.MSELoss()

# Optimizador: Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Example forward pass with random input
inputs = torch.randn(8, 3, 224, 224)  # Batch of 8 images, 3 channels (RGB), 224x224 resolution
outputs = model(inputs)  # Outputs will be the predicted yaw and pitch angles

print(outputs)


/home/idk/git/ceia/vpc2_project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/idk/git/ceia/vpc2_project/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/idk/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 54.2MB/s]


tensor([[ 0.0698, -0.9033],
        [-0.3466, -0.9905],
        [-0.2182, -0.3665],
        [ 0.0155, -0.3948],
        [-0.3240,  0.1585],
        [-0.2158, -0.2421],
        [-0.7822, -1.1779],
        [-0.1719, -0.6169]], grad_fn=<AddmmBackward0>)


Dataloader

In [ ]:
train_loader = DataLoader(...)  # Your training dataset
val_loader = DataLoader(...)    # Your validation dataset

Entrenamiento propiamente dicho

In [ ]:
model.fit(train_loader, val_loader, criterion, optimizer, epochs=10, device=device)